In [109]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error



In [110]:
#first, load 4 datasets
#then combine ratings and ratings_test

add1="C:/Users/YUNUS/Downloads/ml2_project/data/u.base"

add2="C:/Users/YUNUS/Downloads/ml2_project/data/u.test"

ratings=pd.read_csv(add1,header=None,names=["userid","filmid","rating","timestamp"], sep="\t")
ratings_test=pd.read_csv(add2,header=None,names=["userid","filmid","rating","timestamp"], sep="\t")



In [111]:
# convert combined_ratings into pivot table

utility=pd.pivot_table(ratings, values='rating',index='userid',columns='filmid',fill_value=np.nan)


np.set_printoptions(precision=3)
print(utility)


filmid  1     2     3     4     5     6     7     8     9     10    ...  1673  \
userid                                                              ...         
1        5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   NaN   
2        4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...   NaN   
3        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
4        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
5        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
6        4.0   NaN   NaN   NaN   NaN   NaN   2.0   4.0   4.0   NaN  ...   NaN   
7        NaN   NaN   NaN   5.0   NaN   NaN   5.0   5.0   5.0   4.0  ...   NaN   
8        NaN   NaN   NaN   NaN   NaN   NaN   3.0   NaN   NaN   NaN  ...   NaN   
9        NaN   NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN  ...   NaN   
10       4.0   NaN   NaN   4.0   NaN   NaN   NaN   NaN   4.0   NaN  ...   NaN   
11       NaN   NaN   NaN   N

In [112]:
"""""

when predicting a rating for a film given by a user, consider all users rating this film,
and then take an average of all those ratings

"""""
means=utility.mean(axis=1).values
pred_=[]
for rownumber, row in ratings_test.iterrows():
    try:
        pred_.append(avgs[row['filmid']])
    except:
        
        pred_.append(means[row['userid']-1])
    
print("Mean squared error:", mean_squared_error(pred_,ratings_test.rating))


Mean squared error: 1.088130552276007


In [113]:
"""""

Consider the ratings x_1,...,x_n given by an arbitrarily chosen user. Normalize these, that is,
subtract the average (x_1+...+x_n)/n from the ratings. This tells us whether a user rates the movie above the average
movie he has rated or below that average. Normalize the ratings of each user. The remaining is the same as the above.

"""""


user_avgs=ratings.groupby('userid')['rating'].mean()
ratings['diff']=ratings[['userid','rating']].apply(lambda x:x[1]-user_avgs[x[0]],axis=1)
contributions=ratings.groupby('filmid')['diff'].mean()

pred_=[]
for rownumber, row in ratings_test.iterrows():
    try:
        pred_.append(means[row['userid']-1]+contributions[row['filmid']])
    except:
        
        pred_.append(means[row['userid']-1])
    
print("Mean squared error:", mean_squared_error(pred_,ratings_test.rating))

Mean squared error: 0.9637031816291289


In [114]:
"""

Instead of taking average over all users rating a film in question, we only consider 
the most similar users to a user we want to predict. We  measure
similarity of users  via the Pearson Correlation formula.
 
"""



    



# Finds the Pearson Correlation Similarity Measure between two users
#This Pearson correlation and Pandas' Pearson correlation are a bit different

def pcs(x, rowx, y,rowy):
    
    corr=0
    var_x=0
    var_y=0
    for a,b in zip(x,y):
        if a>0 and b>0:
            corr+=((a-means[rowx])*(b-means[rowy]))
            var_x+=(a-means[rowx])**2
            var_y+=(b-means[rowy])**2
    if corr==0:
        return 0
    else:
        return corr/np.sqrt(var_x*var_y)
                

# Guesses the ratings that user with id, user_id, might give to item with id, i_id.
# We will consider the top_n similar users to do this. 
def guess(user_id, i_id, top_n):
    
    list_of_corr=list()
    for row_number in range(utility.shape[0]):
        if row_number!=user_id-1 and  utility.iloc[row_number,i_id-1]>0:
            list_of_corr.append((row_number,pcs(utility.iloc[user_id-1,:],user_id-1,utility.iloc[row_number,:],row_number)))
    list_of_corr.sort(key=lambda x:x[1],reverse=True)
    
    user_guess=means[user_id-1]
    
    for row_number, corr in list_of_corr[:top_n]:
        user_guess+=((utility.iloc[row_number,i_id-1]-means[row_number])/min(len(list_of_corr),top_n))
    
    
    return user_guess





n = 10 # Assume top_n users
import time
start_time = time.time()
    
pred_=[]
for rownumber, row in ratings_test.iterrows():
    pred_.append(guess(row['userid'],row['filmid'],10))
    
print("Mean squared error:", mean_squared_error(pred_,ratings_test.rating))

print("--- %s seconds ---" % (time.time() - start_time))






Mean squared error: 0.9874174178050048
--- 1253.2335002422333 seconds ---


The idea averaging over the most similar users seems promising, but does not improve the result coming from the second case.
Why? It is difficult to detect the similarity between items and users because we have little information about user-item pairs in this sparse utility matrix. 